In [ ]:
import os
import tempfile
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, max as spark_max, min as spark_min, stddev, when, desc, round as spark_round
from pyspark.conf import SparkConf
from pyspark import SparkContext
from minio import Minio
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Inicializar SparkSession
try:
    sc = SparkContext.getOrCreate()
    sc.stop()
except:
    pass
import time
time.sleep(1)
conf = SparkConf().setAppName("GenerarKPIs").setMaster("local[*]").set("spark.driver.bindAddress", "127.0.0.1").set("spark.driver.host", "127.0.0.1")
try:
    sc = SparkContext(conf=conf)
    spark = SparkSession(sc)
except:
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("✅ Spark iniciado")

In [ ]:
# Configuración MinIO
MINIO_ENDPOINT = os.environ.get("MINIO_ENDPOINT", "localhost:9000")
MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY", "minioadmin")
MINIO_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY", "minioadmin")
MINIO_BUCKET_SILVER = "proyecto-silver"
MINIO_BUCKET_GOLD = "proyecto-gold"

minio_client = Minio(MINIO_ENDPOINT, access_key=MINIO_ACCESS_KEY, secret_key=MINIO_SECRET_KEY, secure=False)
print("✅ MinIO conectado")

# Crear bucket Gold si no existe
try:
    minio_client.make_bucket(MINIO_BUCKET_GOLD)
    print(f'✅ Bucket {MINIO_BUCKET_GOLD} creado')
except:
    print(f'✅ Bucket {MINIO_BUCKET_GOLD} ya existe')

In [ ]:
# Cargar datos de Silver
print("\n📥 CARGANDO DATOS DE SILVER...")

try:
    print("Buscando archivos Silver...")
    objects = minio_client.list_objects(MINIO_BUCKET_SILVER, recursive=True)
    archivos_parquet = [obj.object_name for obj in objects if obj.object_name.endswith(".parquet")]
    
    if archivos_parquet:
        dfs = []
        for archivo in sorted(archivos_parquet):
            temp_dir = tempfile.gettempdir()
            temp_file = os.path.join(temp_dir, archivo.split("/")[-1])
            minio_client.fget_object(MINIO_BUCKET_SILVER, archivo, temp_file)
            df_temp = spark.read.parquet(temp_file)
            dfs.append(df_temp)
            print(f"  Cargado: {archivo}")
        
        from functools import reduce
        spark_df = reduce(lambda x, y: x.union(y), dfs)
        print(f"\n✅ Datos consolidados: {spark_df.count()} registros")
    else:
        print("⚠️ Sin archivos en Silver")
        spark_df = spark.createDataFrame([], "id INT, temperature DOUBLE, humidity DOUBLE")
except Exception as e:
    print(f"⚠️ Error: {e}")
    spark_df = spark.createDataFrame([], "id INT, temperature DOUBLE, humidity DOUBLE")

print(f"\n📊 DataFrame: {spark_df.count()} registros")

In [ ]:
# Generar KPIs
print("\n📊 GENERANDO KPIs...")

if spark_df.count() > 0:
    # Calcular KPIs generales
    kpis = {}
    
    kpis['total_readings'] = spark_df.count()
    kpis['unique_sensors'] = spark_df.select('sensor_id').distinct().count() if 'sensor_id' in spark_df.columns else 0
    
    # KPIs de temperatura
    if 'temperature' in spark_df.columns:
        temp_stats = spark_df.select(
            avg('temperature').alias('temp_avg'),
            spark_max('temperature').alias('temp_max'),
            spark_min('temperature').alias('temp_min'),
            stddev('temperature').alias('temp_std')
        ).collect()[0]
        
        kpis['temperature_avg'] = temp_stats['temp_avg']
        kpis['temperature_max'] = temp_stats['temp_max']
        kpis['temperature_min'] = temp_stats['temp_min']
        kpis['temperature_std'] = temp_stats['temp_std']
    
    # KPIs de humedad
    if 'humidity' in spark_df.columns:
        humidity_stats = spark_df.select(
            avg('humidity').alias('humidity_avg'),
            spark_max('humidity').alias('humidity_max'),
            spark_min('humidity').alias('humidity_min')
        ).collect()[0]
        
        kpis['humidity_avg'] = humidity_stats['humidity_avg']
        kpis['humidity_max'] = humidity_stats['humidity_max']
        kpis['humidity_min'] = humidity_stats['humidity_min']
    
    # KPIs de PM2.5
    if 'pm25' in spark_df.columns:
        pm25_stats = spark_df.select(
            avg('pm25').alias('pm25_avg'),
            spark_max('pm25').alias('pm25_max')
        ).collect()[0]
        
        kpis['pm25_avg'] = pm25_stats['pm25_avg']
        kpis['pm25_max'] = pm25_stats['pm25_max']
        
        # Clasificación de calidad del aire
        pm25_val = pm25_stats['pm25_avg']
        if pm25_val > 150:
            kpis['air_quality_category'] = 'Peligroso'
        elif pm25_val > 100:
            kpis['air_quality_category'] = 'Muy insalubre'
        elif pm25_val > 50:
            kpis['air_quality_category'] = 'Insalubre'
        elif pm25_val > 25:
            kpis['air_quality_category'] = 'Moderado'
        else:
            kpis['air_quality_category'] = 'Bueno'
    
    # KPIs de presión
    if 'pressure' in spark_df.columns:
        pressure_stats = spark_df.select(
            avg('pressure').alias('pressure_avg'),
            spark_max('pressure').alias('pressure_max'),
            spark_min('pressure').alias('pressure_min')
        ).collect()[0]
        
        kpis['pressure_avg'] = pressure_stats['pressure_avg']
        kpis['pressure_max'] = pressure_stats['pressure_max']
        kpis['pressure_min'] = pressure_stats['pressure_min']
    
    # Generar alertas
    alerts = {}
    
    if 'temperature' in spark_df.columns:
        alerts['high_temperature'] = spark_df.filter(col('temperature') > 30).count()
        alerts['low_temperature'] = spark_df.filter(col('temperature') < 5).count()
    
    if 'humidity' in spark_df.columns:
        alerts['high_humidity'] = spark_df.filter(col('humidity') > 80).count()
        alerts['low_humidity'] = spark_df.filter(col('humidity') < 20).count()
    
    kpis['alerts_json'] = str(alerts)
    
    print(f"KPIs generados:")
    for key, value in kpis.items():
        if isinstance(value, float):
            print(f"   {key}: {value:.2f}")
        else:
            print(f"   {key}: {value}")
else:
    print("⚠️ No hay datos para generar KPIs")

In [ ]:
# Estadísticas por sensor
print("\n📈 ESTADÍSTICAS POR SENSOR...")

if spark_df.count() > 0 and 'sensor_id' in spark_df.columns:
    sensor_stats = spark_df.groupBy('sensor_id').agg(
        count('*').alias('readings_count'),
        avg('temperature').alias('temp_avg'),
        spark_max('temperature').alias('temp_max'),
        spark_min('temperature').alias('temp_min'),
        avg('humidity').alias('humidity_avg')
    ).orderBy(desc('readings_count'))
    
    print("\nEstadísticas por sensor:")
    sensor_stats.show()
    
    # Convertir a DataFrame de Pandas para guardar
    sensor_stats_pd = sensor_stats.toPandas()
else:
    print("⚠️ No hay datos de sensores para procesar")
    sensor_stats_pd = pd.DataFrame()

In [ ]:
# Guardar KPIs en Gold
print("\n💾 GUARDANDO KPIs EN GOLD...")

try:
    import io
    from pyspark.sql.types import TimestampType
    
    # Guardar KPIs como CSV
    kpis_df = pd.DataFrame([kpis])
    kpis_df['timestamp'] = datetime.now().isoformat()
    kpis_df['generation_time'] = datetime.now()
    
    # Reordenar columnas
    cols = ['timestamp', 'generation_time'] + [c for c in kpis_df.columns if c not in ['timestamp', 'generation_time']]
    kpis_df = kpis_df[cols]
    
    kpis_csv = kpis_df.to_csv(index=False)
    kpis_bytes = io.BytesIO(kpis_csv.encode('utf-8'))
    
    minio_client.put_object(
        MINIO_BUCKET_GOLD,
        "sensor_kpis/kpis.csv",
        kpis_bytes,
        length=len(kpis_csv.encode('utf-8')),
        content_type="text/csv"
    )
    print("✅ KPIs guardados: sensor_kpis/kpis.csv")
    
    # Guardar estadísticas por sensor
    if not sensor_stats_pd.empty:
        sensor_csv = sensor_stats_pd.to_csv(index=False)
        sensor_bytes = io.BytesIO(sensor_csv.encode('utf-8'))
        
        minio_client.put_object(
            MINIO_BUCKET_GOLD,
            "sensor_kpis/sensor_statistics.csv",
            sensor_bytes,
            length=len(sensor_csv.encode('utf-8')),
            content_type="text/csv"
        )
        print("✅ Estadísticas guardadas: sensor_kpis/sensor_statistics.csv")
    
except Exception as e:
    print(f"❌ Error al guardar: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# Log de ejecución
print("\n📝 GUARDANDO LOG DE EJECUCIÓN...")

try:
    log_data = {
        'script': 'gold_kpis_analysis_script',
        'execution_time': datetime.now().isoformat(),
        'kpis_generated': len(kpis),
        'sensors_analyzed': kpis.get('unique_sensors', 0),
        'output_files': ['kpis.parquet', 'sensor_statistics.parquet'],
        'temperature_avg': kpis.get('temperature_avg', None),
        'air_quality': kpis.get('air_quality_category', 'N/A'),
        'total_sensors': kpis.get('unique_sensors', 0),
        'status': 'completed'
    }
    
    log_df = pd.DataFrame([log_data])
    log_csv = log_df.to_csv(index=False)
    log_bytes = io.BytesIO(log_csv.encode('utf-8'))
    
    minio_client.put_object(
        MINIO_BUCKET_GOLD,
        "logs/execution_log.csv",
        log_bytes,
        length=len(log_csv.encode('utf-8')),
        content_type="text/csv"
    )
    
    print("✅ Log guardado exitosamente")
    
except Exception as e:
    print(f"Error al guardar log: {e}")

In [ ]:
# Resumen final
print("\n" + "="*70)
print("✅ PROCESAMIENTO DE GOLD COMPLETADO")
print("="*70)

try:
    # Listar archivos en Gold
    gold_files = minio_client.list_objects(MINIO_BUCKET_GOLD, recursive=True)
    gold_files_list = [obj.object_name for obj in gold_files]
    
    print(f"Procesamiento completado!")
    print(f"Total archivos en Gold: {len(gold_files_list)}")
    print(f"Temperatura promedio: {kpis.get('temperature_avg', 'N/A'):.2f}°C" if isinstance(kpis.get('temperature_avg'), (int, float)) else f"Temperatura promedio: N/A")
    print(f"Calidad del aire: {kpis.get('air_quality_category', 'N/A')}")
    print(f"Sensores únicos: {kpis.get('unique_sensors', 'N/A')}")
except Exception as e:
    print(f"Error en resumen: {e}")

print("="*70)

In [ ]:
# Limpiar SparkContext
print("\n🔚 CERRANDO SESIÓN...")

try:
    if 'spark' in locals():
        spark.stop()
        print("✅ Sesión de Spark cerrada correctamente")
except Exception as e:
    print(f"⚠️ Error al cerrar Spark: {e}")

In [ ]:
# Resumen ejecutivo de KPIs
if not kpis_df.empty and df_silver is not None:
    print("=== RESUMEN EJECUTIVO ===")
    print("\n" + "="*60)
    print("  📈 REPORTE DE KPIs - SISTEMA DE MONITOREO DE SENSORES")
    print("="*60)
    
    # Fecha y hora del reporte
    report_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"📅 Generado: {report_time}")
    
    # Información general
    total_readings = len(df_silver)
    unique_sensors = df_silver['sensor_id'].nunique()
    avg_quality = df_silver['data_quality_score'].mean()
    
    print(f"\n📊 MÉTRICAS GENERALES:")
    print(f"  • Total de lecturas procesadas: {total_readings:,}")
    print(f"  • Sensores activos: {unique_sensors}")
    print(f"  • Calidad promedio de datos: {avg_quality:.1f}/100")
    
    # Condiciones ambientales promedio
    avg_temp = df_silver['temperature'].mean()
    avg_humidity = df_silver['humidity'].mean()
    avg_pressure = df_silver['pressure'].mean()
    
    print(f"\n🌡️  CONDICIONES AMBIENTALES:")
    print(f"  • Temperatura promedio: {avg_temp:.1f}°C")
    print(f"  • Humedad promedio: {avg_humidity:.1f}%")
    print(f"  • Presión promedio: {avg_pressure:.1f} hPa")
    
    # Alertas y anomalías
    if 'total_anomalies' in locals():
        print(f"\n⚠️  ALERTAS:")
        print(f"  • Anomalías detectadas: {total_anomalies}")
        print(f"  • Tasa de anomalías: {anomaly_rate:.2f}%")
    
    # Estado del sistema
    high_quality_readings = (df_silver['data_quality_score'] >= 90).sum()
    quality_percentage = (high_quality_readings / total_readings) * 100
    
    print(f"\n✅ ESTADO DEL SISTEMA:")
    print(f"  • Lecturas de alta calidad: {high_quality_readings:,} ({quality_percentage:.1f}%)")
    
    if quality_percentage >= 95:
        system_status = "🟢 EXCELENTE"
    elif quality_percentage >= 80:
        system_status = "🟡 BUENO"
    else:
        system_status = "🔴 REQUIERE ATENCIÓN"
    
    print(f"  • Estado general: {system_status}")
    
    print("\n" + "="*60)
    print("📁 KPIs disponibles para descarga en capa Gold")
    print("="*60)

In [ ]:
# Exportar resultados
if not kpis_df.empty:
    print("=== EXPORTANDO RESULTADOS ===")
    
    # Exportar KPIs para uso en otros módulos
    export_path = gold_layer.export_kpis_for_module()
    
    if export_path:
        print(f"✅ KPIs exportados exitosamente")
        print(f"📁 Ubicación: {export_path}")
        print(f"📊 Métricas incluidas: {len(kpis_df)}")
        
        # Mostrar resumen de archivos en Gold
        gold_files_updated = minio_client.list_objects(Config.GOLD_PATH)
        export_files = [f for f in gold_files_updated if 'export' in f]
        
        print(f"\n📂 Archivos de exportación disponibles: {len(export_files)}")
        for export_file in export_files[-3:]:  # Mostrar últimos 3
            print(f"  • {export_file}")
            
    else:
        print("❌ Error al exportar KPIs")

# Información de acceso a los datos
print("\n" + "="*50)
print("📋 INFORMACIÓN DE ACCESO A DATOS:")
print("="*50)
print(f"🗂️  Bucket MinIO: {Config.MINIO_BUCKET}")
print(f"📁 Ruta Gold: {Config.GOLD_PATH}")
print(f"🌐 MinIO Console: http://{Config.MINIO_ENDPOINT.replace(':9000', ':9090')}")
print(f"👤 Usuario: {Config.MINIO_ACCESS_KEY}")
print("="*50)

In [ ]:
# Guardar todos los KPIs en capa Gold (archivos únicos que se sobrescriben)
print("=== GUARDANDO EN CAPA GOLD (ARCHIVOS ÚNICOS) ===")

# Crear un diccionario con todos los KPIs calculados
from datetime import datetime
timestamp = datetime.now()

kpis_data = {
    'timestamp': timestamp.isoformat(),
    'generation_time': timestamp,
    'temperature_avg': float(temperature_avg),
    'temperature_max': float(temperature_max),
    'temperature_min': float(temperature_min),
    'humidity_avg': float(humidity_avg),
    'humidity_max': float(humidity_max),
    'humidity_min': float(humidity_min),
    'pm25_avg': float(pm25_avg),
    'pm25_max': float(pm25_max),
    'air_quality_category': air_quality_category,
    'total_sensors': int(total_sensors),
    'total_readings': int(total_readings)
}

# Convertir a DataFrame
kpis_df = pd.DataFrame([kpis_data])

# Guardar KPIs principales (archivo único que se sobrescribe)
kpis_path = f"{Config.GOLD_PATH}/kpis.parquet"

print(f"Sobrescribiendo archivo: {kpis_path}")
success = minio_client.upload_dataframe_as_parquet(kpis_df, kpis_path)

if success:
    print(f"✅ KPIs guardados exitosamente en capa Gold")
    
    # Guardar también los datos completos con estadísticas por sensor
    if 'sensor_stats' in locals():
        sensor_stats_path = f"{Config.GOLD_PATH}/sensor_statistics.parquet"
        stats_success = minio_client.upload_dataframe_as_parquet(sensor_stats, sensor_stats_path)
        if stats_success:
            print(f"✅ Estadísticas por sensor guardadas: {sensor_stats_path}")
    
    # Crear log de ejecución (archivo único que se sobrescribe)
    notebook_marker = {
        'notebook': 'gold_kpis_analysis',
        'execution_time': timestamp.isoformat(),
        'kpis_generated': len(kpis_data) - 2,  # Excluyendo timestamp y generation_time
        'output_files': ['kpis.parquet', 'sensor_statistics.parquet'] if 'sensor_stats' in locals() else ['kpis.parquet'],
        'temperature_avg': float(temperature_avg),
        'air_quality': air_quality_category,
        'total_sensors': int(total_sensors),
        'status': 'completed'
    }
    
    log_df = pd.DataFrame([notebook_marker])
    log_path = f"{Config.GOLD_PATH}/execution_log.parquet"
    
    # Sobrescribir log de ejecución
    minio_client.upload_dataframe_as_parquet(log_df, log_path)
    
    # Mostrar archivos en Gold
    gold_files = minio_client.list_objects(Config.GOLD_PATH)
    
    print("\n=== PROCESO COMPLETADO ===")
    print(f"📊 KPIs generados: {len(kpis_data) - 2}")
    print(f"🏷️  Archivos en Gold: {len(gold_files)}")
    print(f"📈 Temperatura promedio: {temperature_avg:.2f}°C")
    print(f"🌡️  Calidad del aire: {air_quality_category}")
    print(f"📡 Total sensores: {total_sensors}")
    print(f"📝 Log de ejecución actualizado")
    
    print("\n🎉 Análisis de KPIs finalizado exitosamente!")
    print("\n📈 Los KPIs han sido generados y están disponibles en:")
    print(f"   • Capa Gold de MinIO: {Config.GOLD_PATH}")
    print(f"   • Archivo principal: kpis.parquet")
    print(f"   • Estadísticas por sensor: sensor_statistics.parquet")
    print("\n🔄 Este análisis se ejecuta automáticamente según la programación del ETL")
    print("📊 Los KPIs se actualizan con cada nuevo lote de datos procesado")
    print("\n✨ ¡Datos listos para consumo por otros módulos del sistema!")

else:
    print("❌ Error al guardar KPIs en capa Gold")